In [ ]:
from pathlib import Path
import pandas as pd

data_dir = Path("/home/sean/Projects/streambt/full_trade_opportunities.parquet/")
full_df = pd.concat(
    pd.read_parquet(parquet_file)
    for parquet_file in data_dir.glob('*.parquet')
)

In [ ]:
full_df

# profiling strategy overall

In [ ]:
import duckdb as db
db.sql(
"""
select 
    "year(Date)"
    , sum("total trade count") as opportunities
    , sum("trade won") as success
    , sum("trade loss") as fail
    , success/fail
    , sum("average trade return" * "total trade count")/opportunities as "average return rate"
from full_df
group by "year(Date)"
order by "year(Date)" 
"""
).df()

# profiling for each individual stock v2

## next step:
- convert this to pyspark
- parametrize things like 
  - years of strategy consistency
  - max profit margin per trade
  - max loss per trade
- increase grain on performance analysis into trade opportunity level. (try to catch extreme gain events?)
- strategy agnostic metrics like past performance metrics like yearly return on hold to stock
- improve tmf signal with using percentile on strength of dd
- currently tallying win loss ratio of single instrument, change to focus on ranking product of all trades?

In [ ]:
import duckdb as db
win_loss_ratio_ordered = db.sql(
"""
with stg1 as (
    select 
        *
        , "trade won"/"trade loss" as win_loss_ratio
        --, dense_rank() over (partition by "year(Date)" order by win_loss_ratio desc) as drnk
        --, lag("total trade count",1) over (partition by Ticker order by "year(Date)" ) as prev_year_total_trade_count
        --, lag(win_loss_ratio,1) over (partition by Ticker order by "year(Date)" ) AS prev_year_win_loss_ratio
    from full_df
)
,stg2 as (
    select *
        , list(win_loss_ratio order by "year(Date)" ) over (partition by Ticker order by "year(Date)" rows between 5 preceding and 1 preceding) as collated_history
        , cast(list_reduce(collated_history, (acc, x) -> case when (x <= 1) or (acc <= 1) then 1 else 2 end)-1 as boolean) 
            and (len(collated_history)=5) 
        as all_more_win_than_loss
    from stg1
)
select *
from stg2
where all_more_win_than_loss = True
order by "year(Date)"
"""
)

display(db.sql("""select * from win_loss_ratio_ordered where collated_history is not NULL order by "average trade return" """))

binning_filtered_instruments_year_by_wl_ratio =  db.sql(
"""
select  
    case 
        when win_loss_ratio < 1 then 'less than 1' 
        when win_loss_ratio < 2 then 'less than 2' 
        else 'more than 2' 
    end as category
    , count(*) as tally
from win_loss_ratio_ordered
group by category
order by category
"""
)
#display(binning_filtered_instruments_year_by_wl_ratio.df())
import plotly.express as px
px.bar(binning_filtered_instruments_year_by_wl_ratio.df(), x="category", y='tally')


# profiling for each individual stock

In [ ]:
import duckdb as db
win_loss_ratio_ordered = db.sql(
"""
    select 
        *
        , "trade won"/"trade loss" as win_loss_ratio
        , dense_rank() over (partition by "year(Date)" order by win_loss_ratio desc) as drnk
    from full_df
"""
)
# introduces forward pollution
ticker_with_good_year_stat = db.sql(
"""
    select * 
    from win_loss_ratio_ordered
    where 
        "average trade return" >= 1.05 
        and win_loss_ratio > 2 
        and "year(Date)" <= 2020 
        -- and drnk < 10 
    --order by "year(Date)", win_loss_ratio desc
"""
) 
# verify tickers with good occurrance count
occ_of_ticker = db.sql(
"""
    select count(*) as occurrance, Ticker
    from ticker_with_good_year_stat
    group by Ticker
    --order by occurrance desc
"""
)
# subsequent binning
binning = db.sql(
"""
select count(*) as val, occurrance
from occ_of_ticker
group by occurrance
order by occurrance asc
"""
).df()

filter_for_consistently_good_ticker = db.sql("""select * from occ_of_ticker where occurrance >= 10 order by occurrance desc""")
print("cnt of consistently good tickers\n", db.sql("select count(*) from filter_for_consistently_good_ticker"))

# verify the yoy performance of these stocks
filter_performance_verification = db.sql(
"""
with base as (
    select *
    from win_loss_ratio_ordered t
    inner join filter_for_consistently_good_ticker f on t.Ticker = f.Ticker 
    where "year(Date)" > 2020 -- dont validate on reference data
    order by "year(Date)", win_loss_ratio desc
)
select a.*
    , b."total trade count" as prev_year_total_trade_count
    , b.win_loss_ratio AS prev_year_win_loss_ratio
from base a
left join base b on 
    a.Ticker = b.Ticker 
    and a."year(Date)" = (b."year(Date)" + 1)
where b.win_loss_ratio > 1
ORDER BY a.Ticker, a."year(Date)"
"""
)

import plotly.express as px
ax = px.bar(binning,x='occurrance',y='val')
ax.show()
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    #display(filter_for_consistently_good_ticker.df())
    display(filter_performance_verification.df())
    pass

binning_filtered_instruments_year_by_wl_ratio =  db.sql(
"""
select  
    case 
        when win_loss_ratio < 1 then 'less than 1' 
        when win_loss_ratio < 2 then 'less than 2' 
        else 'more than 2' 
    end as category
    , count(*) as tally
from filter_performance_verification
group by category
order by category
"""
)
#display(binning_filtered_instruments_year_by_wl_ratio.df())
px.bar(binning_filtered_instruments_year_by_wl_ratio.df(), x="category", y='tally')

In [ ]:
db.sql(
"""
    select * 
    from win_loss_ratio_ordered
    where Ticker = 'CBA.AX'
    order by "year(Date)", win_loss_ratio desc
"""
).df()